<img src="Wallaby.png"/>

## Project 2: Extracting the kinematics of galaxies from hyperspectral cubes in order to model rotation curves in the age of the SKA

---

In this project we aim to meet the following criteria:

- Model [HI disks](https://www.skatelescope.org/galaxyevolution/) of galaxies as interferometric data cubes
- Return information from the data cubes necessary for modelling the HI disk rotation curves (i.e. radius vs velocity)
- Explore the ability of making this a self-supervised method using [pysics aware neural networks](https://arxiv.org/abs/1907.03957)
- Test our models on the release of [WALLABY](https://wallaby-survey.org/) (a pathfinder of SKA) data in the next few months

---

### Why is it important to study HI in galaxies?

- HI disks tend to extend far out beyond the molecular gas component making it an ideal tracer of the inter galactic medium 
- Being hotter, less dense, and extends to higher potential energy regions than molecular gas, HI is more succeptible to dynamical events such as ram pressure stripping, making it an ideal way to study galaxy kinematics
- Given the extent of HI, it can be used to trace the density of the dark matter halo within which the galaxy lies (see [Image](https://upload.wikimedia.org/wikipedia/commons/thumb/c/cd/Rotation_curve_of_spiral_galaxy_Messier_33_%28Triangulum%29.png/970px-Rotation_curve_of_spiral_galaxy_Messier_33_%28Triangulum%29.png))

---

### The prototype plans so far:

So far we have created been using [KinMS](https://github.com/TimothyADavis/KinMSpy) to create fake datacubes of galaxies (dimensions being: spatial, spatial, frequency). These galaxies are modelled as simple [exponential disks](https://en.wikipedia.org/wiki/Galactic_disc), with varying scale lengths, [position angle](https://en.wikipedia.org/wiki/Position_angle), [inclination angle](http://hosting.astro.cornell.edu/academics/courses/astro201/gal_inclination.htm), and velocity profile. We have not varied the morphology (i.e. all galaxies are disks with no spiral arms) and all galaxies are perfectly centered in the image plane.

We have trained a simple 3 layer CNN to try and recover the following:

- Position angle $\theta_{\text{pos}}$
- Inclination $\phi_{\text{inc}}$
- Exponential scale length $a$
- Maximum line of sight velocity: $\text{V}\sin(\phi_{\text{inc}})$

The results of a simple true versus prediction of these properties can be seen below. (As you can see I've put a lot of effort into getting $\theta_{pos}$ by learning the components of an Euler rotation matrix in order to prevent striping at the discontinuous $0^{\circ}$/$360^{\circ}$ point. Maybe there are ways to dive into the cube and find better ways of learning the other relevant parameters..

At this stage it's more of a proof of  concept that you can in fact extract information from the raw cubes using neural networks. We were able to show that using cubes over 2D images (like in our first published paper) yields higher accuracy which is good news! But we have no feel for returning velocity characteristics yet.   

Note: For the final model we won't actually care about the scale length $a$ but it's nice to see how well the model can return that property from a data cube. 

<img src="cube_velocity_test.png" width="700"/>

Our ultimate goal is to get a full rotation curve, i.e. rather than returning $\text{V}\sin(\phi_{\text{inc}})$ we want to return $\text{V}(\text{r})\sin(\phi_{\text{inc}})$. I believe there are two ways to do this:


**1. Force the rotation curve to be constrained to one model and learn the best parameters to fit that model**

**2. Force the galaxy to have a finite number of radii over which to evaluate the line of sight velocity and allow the model to fit those velocities**

Option 1 is my favourite as it allows us to try both supervised and self-supervised approaches under the obvious constraint of assuming all galaxy rotation curves follow one generic model. Such a model would look like that shown below.

<img src="AE.png" width="500"/>

In this network the decoder is an evaluation of a known function, written in native $\texttt{PyTorch}$ (so that gradients can be tracked), for every node in the resulting data cube. Under the constraint of $\text{g}(\xi)$ modelling the entirety of the input data cube, we can learn the parameters $\xi_{i}$ that describe the velocity profile $\text{V}\sin(\phi_{\text{inc}})$ in a self-supervised way...hopefully.

### Information regarding the decoder model

If we want to evaluate the output cube on a pixel by pixel basis we must constrain the surface brightness and the velocity profile. 

A simple model for the surface brightness profile is an exponential disk governed by the following formula:

$\text{I}(\text{r}) = \text{I}_{0}\exp{(-\frac{r}{a})}$,

Where $\text{r}$ denotes the radius and a denotes the exponential scale length.

And the velocity function takes the form:

$\text{V}^{2}(\text{r}) = \text{V}_{\text{h}}^{2}\left(1-\frac{a_{h}}{\text{r}}\tan^{-1}\left(\frac{r}{a_{h}}\right)\right)$,

Where $\text{a}_{\text{h}}$ is the scale length of the dark matter halo and $\text{V}_{\text{h}}$ is the velocity at which the HI disk asymptotes to at high $\text{r}$.

Once the encoder makes a prediction on all relevant parameters for the disk, we simply then calculate the radii of pixels in the transformed disk and apply the two formulae above to generate the cube. The difficulty is in assigning pixels to their correct channels in a fast and efficient manner (I don't believe PyTorch functions are needed for this as it's the velocity profile parameters that govern the channels they fall into anyway).

<img src="channels.gif" width="700"/>